In [60]:
import sys
sys.path.append("../")

import subprocess
import datetime
from joblib import Parallel, delayed

import json
import os

import matplotlib.pyplot as plt
from tqdm import tqdm
import optuna

from run import *

In [61]:
start_case = 0
case_num = 1000

cases = [
    (f"tools/in/{seed:04}.txt", f"tools/out/{seed:04}.txt")
    for seed in range(start_case, start_case + case_num)
]


def objective(trial: optuna.trial.Trial) -> float:
    min_k = 4
    max_k = 5.5
    k_p = 0.85
    start_step = 0.75
    end_step = 1.8
    step_cnt = 5
    v_prob_mean_w = 1.460
    v_prob_min_var = 0.027
    p_max = 0.9
    p_min = 0.4
    delta_max_dist = 2
    neighbor_p = 2.
    duplicate_min_d = 1
    step_p = 1.
    out_step_size = 100.
    start_temp_coef = 1e3
    end_temp_coef = 1e5

    # min_k = trial.suggest_float("max_k", low=1.5, high=4.5)
    # max_k = trial.suggest_float("min_k", low=5.0, high=7.0)
    # k_p = trial.suggest_float("k_p", low=0.5, high=2.0)
    # start_step = trial.suggest_float("start_step", low=0.2, high=1.5)
    # end_step = trial.suggest_float("end_step", low=max(1.2, start_step + 0.1), high=3.2)
    # step_cnt = trial.suggest_int("step_cnt", low=2, high=12)
    # v_prob_mean_w = trial.suggest_float("v_prob_mean_w", low=0.0, high=3.0)
    # v_prob_min_var = trial.suggest_float("v_prob_min_var", low=0.01, high=0.5)
    # p_max = trial.suggest_float("p_max", low=0.5, high=0.99)
    # p_min = trial.suggest_float("p_min", low=0.1, high=min(0.9, p_max + 0.01))
    # delta_max_dist = trial.suggest_int("delta_max_dist", low=1, high=3)
    # neighbor_p = trial.suggest_float("neighbor_p", low=1., high=3.)
    # duplicate_min_d = trial.suggest_int("duplicate_min_d", low=1, high=5)
    # step_p = trial.suggest_float("step_p", low=0.5, high=2.)
    # out_step_size = trial.suggest_float("out_step_size", low=50., high=200.)
    # start_temp_coef = trial.suggest_float("start_temp_coef", low=1e2, high=1e4)
    # end_temp_coef = trial.suggest_float("end_temp_coef", low=1e4+1, high=1e6)

    action_slide_ratio = trial.suggest_float("action_slide_ratio", low=0.01, high=0.6)
    action_move_one = trial.suggest_float("action_move_one", low=0.01, high=0.6)
    action_swap_2 = 1.0
    action_swap_3 = trial.suggest_float("action_swap_3", low=0.01, high=0.6)

    args = " ".join(
        map(
            str,
            [
                min_k,
                max_k,
                k_p,
                start_step,
                end_step,
                step_cnt,
                v_prob_mean_w,
                v_prob_min_var,
                p_max,
                p_min,
                delta_max_dist,
                neighbor_p,
                duplicate_min_d,
                step_p,
                out_step_size,
                start_temp_coef,
                end_temp_coef,
                action_slide_ratio,
                action_move_one,
                action_swap_2,
                action_swap_3,
            ],
        )
    )

    solver_path = "./target/release/ahc030"
    solver_cmd = f"./tools/target/release/tester {solver_path} {args}"
    solver_version = f"optuna-action-ratio-01-{args}"
    runner = Runner(
        Input,
        Result,
        solver_cmd=solver_cmd,
        solver_version=solver_version,
        database_csv="../log/database.csv",
        log_file="../log/a.log",
        verbose=0,
    )
    runner.run(cases=cases, verbose=0)
    score_df = runner.evaluate_overall_relative_score(
        ignore_solver_prefix="optuna-action-ratio-01-"
    )
    return score_df.relative_score.mean()


subprocess.run(
    "cargo build --features local --release --manifest-path=../Cargo.toml", shell=True
)

study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///ahc030.db",
    study_name="tune-action-ratio-01",
    load_if_exists=True,
)
print("start optimize")
study.enqueue_trial(
    {
        "min_k": 4.0,
        "max_k": 5.5,
        "k_p": 0.85,
        "start_step": 0.75,
        "end_step": 1.8,
        "step_cnt": 5,
        "v_prob_mean_w": 1.0,
        "v_prob_min_var": 0.1,
        "p_max": 0.9,
        "p_min": 0.5,
        "delta_max_dist": 2,
        "neighbor_p": 2.,
        "duplicate_min_d": 1,
        "step_p": 1.0,
        "out_step_size": 100.0,
        "start_temp_coef": 1e3,
        "end_temp_coef": 1e5,
        "action_slide_ratio": 0.2,
        "action_move_one": 0.1,
        "action_swap_2": 0.6,
        "action_swap_3": 0.1,
    }
)
study.optimize(objective, n_trials=100, show_progress_bar=True)
study.best_params

    Finished release [optimized] target(s) in 0.01s
[I 2024-02-19 15:34:01,260] A new study created in RDB with name: tune-action-ratio-01


start optimize


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-02-19 15:36:10,385] Trial 0 finished with value: 0.4712299630948509 and parameters: {'action_slide_ratio': 0.2, 'action_move_one': 0.1, 'action_swap_3': 0.6}. Best is trial 0 with value: 0.4712299630948509.
[I 2024-02-19 15:38:16,989] Trial 1 finished with value: 0.47845499315490864 and parameters: {'action_slide_ratio': 0.3503176416757887, 'action_move_one': 0.20794005713469524, 'action_swap_3': 0.28872183829083603}. Best is trial 1 with value: 0.47845499315490864.
[I 2024-02-19 15:40:20,613] Trial 2 finished with value: 0.49709852423138534 and parameters: {'action_slide_ratio': 0.314306499890376, 'action_move_one': 0.15707224461293465, 'action_swap_3': 0.01932865674934233}. Best is trial 2 with value: 0.49709852423138534.
[I 2024-02-19 15:42:25,482] Trial 3 finished with value: 0.4801834884096679 and parameters: {'action_slide_ratio': 0.0980676109490788, 'action_move_one': 0.20368905316956973, 'action_swap_3': 0.213502716384823}. Best is trial 2 with value: 0.4970985242313853

KeyboardInterrupt: 

In [64]:
study = optuna.load_study(
    storage="sqlite:///ahc030.db",
    study_name="tune-steps-03",
)
study.best_params

{'end_step': 1.4642511974988384,
 'start_step': 0.7476624349317118,
 'step_cnt': 4}

In [73]:
study.trials_dataframe().sort_values("value", ascending=False)[:10]

,number,value,datetime_start,datetime_complete,duration,params_end_step,params_start_step,params_step_cnt,system_attrs_fixed_params,state
196,196,0.563198,2024-02-19 08:33:54.931342,2024-02-19 08:36:20.077629,0 days 00:02:25.146287,1.464251,0.747662,4,NaN,COMPLETE
194,194,0.561573,2024-02-19 08:29:12.693073,2024-02-19 08:31:35.479643,0 days 00:02:22.786570,1.480404,0.805771,4,NaN,COMPLETE
97,97,0.560335,2024-02-19 04:37:35.981380,2024-02-19 04:39:59.795687,0 days 00:02:23.814307,1.544976,0.739561,4,NaN,COMPLETE
166,166,0.560090,2024-02-19 07:24:05.873559,2024-02-19 07:26:31.004545,0 days 00:02:25.130986,1.453658,0.748792,4,NaN,COMPLETE
76,76,0.558605,2024-02-19 03:50:41.240640,2024-02-19 03:52:59.510703,0 days 00:02:18.270063,1.710233,0.711692,4,NaN,COMPLETE
158,158,0.558332,2024-02-19 07:06:47.701353,2024-02-19 07:08:55.242102,0 days 00:02:07.540749,1.574324,0.733187,6,NaN,COMPLETE
107,107,0.557683,2024-02-19 05:04:23.597511,2024-02-19 05:06:42.194884,0 days 00:02:18.597373,1.705467,0.725534,4,NaN,COMPLETE
148,148,0.556817,2024-02-19 06:44:03.591661,2024-02-19 06:46:14.906486,0 days 00:02:11.314825,1.592934,0.735367,5,NaN,COMPLETE
124,124,0.555822,2024-02-19 05:46:26.285890,2024-02-19 05:48:48.113827,0 days 00:02:21.827937,1.676958,0.713187,4,NaN,COMPLETE
96,96,0.555517,2024-02-19 04:35:12.699479,2024-02-19 04:37:35.972674,0 days 00:02:23.273195,1.548320,0.770853,4,NaN,COMPLETE


In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)